In [1]:
from qiskit import Aer, QuantumCircuit, transpile, assemble
from qiskit.visualization import plot_histogram
import numpy as np

In [2]:
# Generate random expected returns for assets
np.random.seed(42)
num_assets = 3
expected_returns = np.random.rand(num_assets)

In [3]:
# Generate a random covariance matrix for the assets
covariance_matrix = np.random.rand(num_assets, num_assets)
covariance_matrix = (covariance_matrix + covariance_matrix.T) / 2  # Ensure symmetry
covariance_matrix[np.arange(num_assets), np.arange(num_assets)] = 1  # Set diagonal to 1 for simplicity

In [4]:
# Define the Quantum Circuit for Portfolio Optimization
def portfolio_optimization_circuit(expected_returns, covariance_matrix):
    num_assets = len(expected_returns)
    num_qubits = 2 * num_assets

    # Create a quantum circuit with the necessary number of qubits
    qc = QuantumCircuit(num_qubits)

    # Apply Hadamard gates to the first set of qubits
    qc.h(range(num_assets))

    # Apply a controlled rotation based on the covariance matrix
    for i in range(num_assets):
        for j in range(i + 1, num_assets):
            qc.cp(2 * np.arcsin(covariance_matrix[i, j]), i, j + num_assets)
            qc.cp(2 * np.arcsin(covariance_matrix[i, j]), j, i + num_assets)

    # Apply a rotation based on the expected returns
    qc.ry(2 * expected_returns[0], 0)
    qc.ry(2 * expected_returns[1], 1)
    qc.ry(2 * expected_returns[2], 2)

    # Measure the qubits
    qc.measure_all()

    return qc


In [5]:
# Create the quantum circuit
qc = portfolio_optimization_circuit(expected_returns, covariance_matrix)

In [ ]:
# Simulate the quantum circuit
simulator = Aer.get_backend('qasm_simulator')
compiled_circuit = transpile(qc, simulator)
qobj = assemble(compiled_circuit)
result = simulator.run(qobj).result()